https://gist.github.com/dirko/1d596ca757a541da96ac3caa6f291229

http://dirko.github.io/Bidirectional-LSTMs-with-Keras/

In [1]:
# author: Keras==1.0.6
# mine: Keras==1.2.1
import numpy as np
import re

from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import TimeDistributedDense, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.layers import Merge
from keras.backend import tf

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.cross_validation import train_test_split

from lambdawithmask import Lambda as MaskLambda

Using TensorFlow backend.
/Users/jzhu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def encode(x, n):
    result = np.zeros(n)
    result[x] = 1
    return result

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        
def reverse_func(x, mask=None):
    return tf.reverse(x, [False, True, False])

def score1(yh, pr):
    coords = [np.where(yhh > 0)[0][0] for yhh in yh]
    yh = [yhh[co:] for yhh, co in zip(yh, coords)]
    ypr = [prr[co:] for prr, co in zip(pr, coords)]
    fyh = [c for row in yh for c in row]
    fpr = [c for row in ypr for c in row]
    return fyh, fpr

In [3]:
raw = open('train.csv', 'r').readlines()
all_x = []
point = []
for line in raw:
    stripped_line = line.strip().split(',')
    point.append(stripped_line)
    if line == '""\r\n':
#         print "newline"
        all_x.append(point[:-1])
        point = []
all_x = all_x[:-1]
lengths = [len(x) for x in all_x]
# short_x = [x for x in all_x if len(x) < 64]

# split long sections into chucks (a mimic of sentences)
short_x = []
for l in all_x:
    short_x.extend(chunks(l, 64))

In [4]:
len(short_x)

3428

In [5]:
X = [[c[0] for c in x] for x in short_x]
y = [[c[1] for c in y] for y in short_x]

In [6]:
all_text = [c for x in X for c in x]
words = list(set(all_text))
word2ind = {word: index for index, word in enumerate(words)}
ind2word = {index: word for index, word in enumerate(words)}
labels = list(set([c for x in y for c in x]))
# label2ind = {label: (index + 1) for index, label in enumerate(labels)}
# ind2label = {(index + 1): label for index, label in enumerate(labels)}
label2ind = {label: (index) for index, label in enumerate(labels)}
ind2label = {(index): label for index, label in enumerate(labels)}
print 'Input sequence length range: ', max(lengths), min(lengths)

Input sequence length range:  4741 58


In [7]:
ind2label

{0: 'Severity',
 1: 'Negation',
 2: 'O',
 3: 'DrugClass',
 4: 'Animal',
 5: 'Factor',
 6: 'AdverseReaction'}

In [8]:
maxlen = max([len(x) for x in X])
print 'Maximum sequence length:', maxlen

Maximum sequence length: 64


In [9]:
X_enc = [[word2ind[c] for c in x] for x in X]
X_enc_reverse = [[c for c in reversed(x)] for x in X_enc]
max_label = max(label2ind.values()) + 1
# max_label = max(label2ind.values())
y_enc = [[0] * (maxlen - len(ey)) + [label2ind[c] for c in ey] for ey in y]
y_enc = [[encode(c, max_label) for c in ey] for ey in y_enc]

In [10]:
max_label

7

In [11]:
y_enc[0][0]

array([ 0.,  0.,  1.,  0.,  0.,  0.,  0.])

In [12]:
X_enc_f = pad_sequences(X_enc, maxlen=maxlen)
X_enc_b = pad_sequences(X_enc_reverse, maxlen=maxlen)
y_enc = pad_sequences(y_enc, maxlen=maxlen)

In [13]:
(X_train_f, X_test_f, X_train_b,
 X_test_b, y_train, y_test) = train_test_split(X_enc_f, X_enc_b, y_enc,
                                               test_size=11*32, train_size=45*32, random_state=42)
print 'Training and testing tensor shapes:'
print X_train_f.shape, X_test_f.shape, X_train_b.shape, X_test_b.shape, y_train.shape, y_test.shape

Training and testing tensor shapes:
(1440, 64) (352, 64) (1440, 64) (352, 64) (1440, 64, 7) (352, 64, 7)


In [14]:
max_features = len(word2ind)
embedding_size = 128
hidden_size = 32
out_size = len(label2ind) #+ 1

In [15]:
model_forward = Sequential()
model_forward.add(Embedding(max_features, embedding_size, input_length=maxlen, mask_zero=True))
model_forward.add(LSTM(hidden_size, return_sequences=True))  

In [16]:
model_backward = Sequential()
model_backward.add(Embedding(max_features, embedding_size, input_length=maxlen, mask_zero=True))
model_backward.add(LSTM(hidden_size, return_sequences=True))
model_backward.add(MaskLambda(function=reverse_func, mask_function=reverse_func))

In [17]:
model = Sequential()

model.add(Merge([model_forward, model_backward], mode='concat'))
model.add(TimeDistributedDense(out_size))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

/Users/jzhu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:1205: UserWarning: `TimeDistributedDense` is deprecated, And will be removed on May 1st, 2017. Please use a `Dense` layer instead.
  warnings.warn('`TimeDistributedDense` is deprecated, '


In [19]:
batch_size = 32
model.fit([X_train_f, X_train_b], y_train, batch_size=batch_size, nb_epoch=100, # started from 40
          validation_data=([X_test_f, X_test_b], y_test))
score = model.evaluate([X_test_f, X_test_b], y_test, batch_size=batch_size)
print('Raw test score:', score)

Train on 1440 samples, validate on 352 samples
Epoch 1/100
1440/1440 [==============================] - 11s - loss: 0.0376 - val_loss: 0.1632
Epoch 2/100
1440/1440 [==============================] - 10s - loss: 0.0364 - val_loss: 0.1669
Epoch 3/100
1440/1440 [==============================] - 10s - loss: 0.0353 - val_loss: 0.1608
Epoch 4/100
1440/1440 [==============================] - 10s - loss: 0.0343 - val_loss: 0.1693
Epoch 5/100
1440/1440 [==============================] - 10s - loss: 0.0334 - val_loss: 0.1587
Epoch 6/100
1440/1440 [==============================] - 10s - loss: 0.0325 - val_loss: 0.1687
Epoch 7/100
1440/1440 [==============================] - 10s - loss: 0.0316 - val_loss: 0.1760
Epoch 8/100
1440/1440 [==============================] - 10s - loss: 0.0309 - val_loss: 0.1745
Epoch 9/100
1440/1440 [==============================] - 10s - loss: 0.0302 - val_loss: 0.1718
Epoch 10/100
1440/1440 [==============================] - 10s - loss: 0.0292 - val_loss: 0.1722
Ep

1440/1440 [==============================] - 10s - loss: 0.0045 - val_loss: 0.1891
Epoch 86/100
1440/1440 [==============================] - 10s - loss: 0.0044 - val_loss: 0.1891
Epoch 87/100
1440/1440 [==============================] - 10s - loss: 0.0043 - val_loss: 0.1892
Epoch 88/100
1440/1440 [==============================] - 10s - loss: 0.0042 - val_loss: 0.1900
Epoch 89/100
1440/1440 [==============================] - 10s - loss: 0.0041 - val_loss: 0.1898
Epoch 90/100
1440/1440 [==============================] - 10s - loss: 0.0040 - val_loss: 0.1898
Epoch 91/100
1440/1440 [==============================] - 10s - loss: 0.0039 - val_loss: 0.1904
Epoch 92/100
1440/1440 [==============================] - 10s - loss: 0.0038 - val_loss: 0.1892
Epoch 93/100
1440/1440 [==============================] - 10s - loss: 0.0037 - val_loss: 0.1914
Epoch 94/100
1440/1440 [==============================] - 10s - loss: 0.0036 - val_loss: 0.1906
Epoch 95/100
1440/1440 [=============================

In [20]:
pr = model.predict_classes([X_train_f, X_train_b])
yh = y_train.argmax(2)
fyh, fpr = score1(yh, pr)
print 'Training accuracy:', accuracy_score(fyh, fpr)
print 'Training confusion matrix:'
print confusion_matrix(fyh, fpr)

1440/1440 [==============================] - 1s     
Training accuracy: 1.0
Training confusion matrix:
[[  367     0     0     0     0     0     0]
 [    0    42     0     0     0     0     0]
 [    0     0 82151     0     0     0     0]
 [    0     0     0    99     0     0     0]
 [    0     0     0     0    14     0     0]
 [    0     0     0     0     0   266     0]
 [    0     0     0     0     0     0  5766]]


In [21]:
pr = model.predict_classes([X_test_f, X_test_b])
yh = y_test.argmax(2)
fyh, fpr = score1(yh, pr)
print 'Testing accuracy:', accuracy_score(fyh, fpr)
print 'Testing confusion matrix:'
print confusion_matrix(fyh, fpr)

352/352 [==============================] - 0s     
Testing accuracy: 0.971455093989
Testing confusion matrix:
[[   51     0    26     0     0     0     5]
 [    0     2    11     0     0     0     0]
 [   42     1 19758     6     0    22   111]
 [    2     0    19    13     0     0     1]
 [    0     0     1     0     2     0     0]
 [    0     0    27     0     0    11     0]
 [   13     0   328     0     0     0  1093]]


In [22]:
ind2label

{0: 'Severity',
 1: 'Negation',
 2: 'O',
 3: 'DrugClass',
 4: 'Animal',
 5: 'Factor',
 6: 'AdverseReaction'}

# Label the predictions

In [27]:
doc = "Classical HL post-auto-HSCT consolidation: neutropenia, peripheral sensory neuropathy, thrombocytopenia, anemia, upper respiratory tract infection, fatigue, peripheral motor neuropathy, nausea, cough, and diarrhea."
x_new = [[m.group(0) for m in re.finditer(r'\w+', doc)]] # tokenize words only

X_new = [[word2ind[c] for c in x] for x in x_new]
X_new_reverse = [[c for c in reversed(x)] for x in X_new]
X_new_f = pad_sequences(X_new, maxlen=maxlen)
X_new_b = pad_sequences(X_new_reverse, maxlen=maxlen)
print X_new_f.shape, X_new_b.shape

(1, 64) (1, 64)


In [24]:
pr = model.predict_classes([X_new_f, X_new_b])
pr

1/1 [==============================] - 0s


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 6, 6, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2,
        2, 2, 2, 6, 6, 6, 6, 6, 2, 2, 2, 2, 6, 6, 2, 2, 2, 6, 2, 2]])

In [39]:
for w, p in zip(x_new[0], pr.tolist()[0][-len(X_new[0]):]):
    print w, ind2label[p]

Classical O
HL O
post O
auto O
HSCT O
consolidation O
neutropenia O
peripheral AdverseReaction
sensory AdverseReaction
neuropathy AdverseReaction
thrombocytopenia AdverseReaction
anemia AdverseReaction
upper O
respiratory O
tract O
infection O
fatigue AdverseReaction
peripheral AdverseReaction
motor O
neuropathy O
nausea O
cough AdverseReaction
and O
diarrhea O


### Issues:
* Cannot identify names with more than one word (how:)
* Two classes are missing in training data (why?)
* How to calculate and improve F1 score? Overfitting?

# Write submission file

https://bionlp.nlm.nih.gov/tac2017adversereactions/